In [ ]:
import sys
sys.path.append('..')
from utilities.utils import extract_frames

extract_frames('../datasets/valencia_open/vide_trimmed2.mp4')

The next cell does all the preprocessing necessary and countour detection and saves the countours coordinates and player positions to `jsonl` files

In [ ]:

import torch
from model_functions import *
import os

model = torch.hub.load(repo_or_dir='/Users/jc/.cache/torch/hub/ultralytics_yolov5_master', model='yolov5s', source='local', device='mps')

directory_path = '../datasets/danish_open_bestpoint/'
filenames = os.listdir(directory_path)
images_paths = sorted(filenames, key=lambda x: int(x.lstrip('frame').rstrip('.png')))
images_paths = [directory_path + image_path for image_path in images_paths]
with open('countours_v2.jsonl', 'w') as f, open('players_position.jsonl', 'w') as  f1:
    improved_get_countours_and_save_results(model=model, imgs=images_paths)


Now the `Resnet50` model runs on all the countour regions to detect the ball.


In [ ]:
# Open the JSONL file and read its contents into a list
import json
import pandas as pd
from model_functions import *
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from model_ball_detection.evaluate import load_means_stds

with open('countours_v2.jsonl', 'r') as f:
    lines = f.readlines()

# Parse each line of the JSONL file into a dictionary
data = []
for line in lines:
    json_data = json.loads(line)
    data.append(json_data)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)
print(df.head())
model_path = 'model_v2.pth'
model =  torch.load(model_path)
if model_path.split('_')[1].split('.')[0] == 'v1':
    image_frame_diff_path = 'image'
    means, stds = load_means_stds(version='v1')
else:
    image_frame_diff_path = 'image_frame_diff_path'
    means, stds = load_means_stds(version='v2')
    
model.eval()
resnet_contours = df.apply(lambda row: predict_resnet(
    model=model, image_path=row['image'],
    image_frame_diff_path=row[image_frame_diff_path],
    countours=row['countours'],
    max_countour=row['maxCountour'],
    players_bboxes=row['players_bboxes'],
    means=means,
    stds=stds,
    ),
    axis='columns'
    )
df['resnet_countours'] = resnet_contours

Code to compute ball directions and the quadrant direction of the ball

In [ ]:
def determine_quadrant(point: dict):
    x = point['x']
    y =  point['y']
    X = 1920
    Y = 1080
    X_half = 1920/2
    Y_half = 1080/2

    # First quadrant
    if x < X_half and y < Y_half:
        quadrant = 1
    # Second quadrant
    elif x > X_half and y < Y_half:
        quadrant = 2
    elif x < X_half and y > Y_half:
        quadrant = 3
    elif x > X_half and y > Y_half:
        quadrant = 4 
    else:
        print(f'point with coordinates {x,y} no quadrant found')
        return None   
    return int(quadrant)

def compute_direction(point1: dict, point2: dict):
    if point1 == None or point2 == None:
        return None
    v1 = point2['x'] - point1['x']
    v2 = point2['y'] - point1['y']
    
    # we don't care about r, just theta, so set r to the same value for all observations, r = 1
    r = 1
    try:
        theta = np.arctan(abs(v2)/abs(v1))
    except ZeroDivisionError:
        return None
    print(v1, v2)
    theta = theta * 360/(2 * np.pi)
    if v2 <= 0 and v1 <= 0:
        theta_r = 180 - theta
    elif v2 >= 0 and v1 <= 0:
        theta_r = 180 + theta
    elif v2 <= 0  and v1 >= 0:
        theta_r = theta
    elif v2 >= 0 and v1 >= 0:
        theta_r = 360 - theta
    return r, theta_r

def clean_consecutive_quadrants(df: pd.DataFrame):
    previous_quadrant = df['quadrant'].iloc[0]
    indices_to_drop = []
    for i in range(1, len(df['quadrant'])):
        current_quadrant = df['quadrant'].iloc[i]
        if current_quadrant == previous_quadrant:
            indices_to_drop.append(i)
        elif (current_quadrant == 1 and previous_quadrant == 2) or (current_quadrant == 2 and previous_quadrant == 1) or (current_quadrant == 3 and previous_quadrant == 4) or (current_quadrant == 4 and previous_quadrant == 3) :
            indices_to_drop.append(i)
        else:
            previous_quadrant = current_quadrant
    return indices_to_drop
    
main_df = df.drop(['countours', 'image_frame_diff_path', 'maxCountour'], axis=1)
main_df = main_df[(main_df.resnet_countours != None) & (  main_df.resnet_countours.notnull())]
main_df = main_df.reset_index()
main_df['quadrant'] = main_df['resnet_countours'].apply(determine_quadrant)
indices = clean_consecutive_quadrants(main_df)
main_df = main_df.drop(index=indices)
main_df_shifted = main_df.shift(1)
main_df_shifted = main_df_shifted.rename(columns={'resnet_countours': 'resnet_countours_shifted'})
main_df_concatenated = pd.concat([main_df, main_df_shifted], axis=1)
print(main_df_concatenated['quadrant'])
directions_df = main_df_concatenated.apply(lambda row: compute_direction(point1=row['resnet_countours'], point2=row['resnet_countours_shifted']), axis=1, result_type='expand')
directions_df = directions_df.dropna()
directions_df = directions_df.apply(lambda x: pd.Series(x))
directions_df.columns = ['r', 'theta']

Code to create the figures for ball positions and directions


In [ ]:
def create_polar_bar_chart_with_transparency(angles_degrees, num_bins, transparency=1, save_path=None):
    bin_boundaries = np.linspace(0, 360, num_bins + 1)

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    hist, bin_edges = np.histogram(angles_degrees, bins=bin_boundaries)

    bin_boundaries_radians = np.radians(bin_boundaries)

    # Plot polar bar chart with transparency and red color
    for i in range(num_bins):
        start_angle = bin_boundaries_radians[i]
        end_angle = bin_boundaries_radians[i + 1]
        ax.bar(x=[start_angle], height=[hist[i]], width=[end_angle - start_angle],
               bottom=0, alpha=transparency, color='red')

    # Set title and save the figure if a save path is provided
    plt.title("Ball directions")
    if save_path:
        plt.savefig(save_path)

    # Show the plot
    plt.show()
def plot_surface_points(df, save_filename=None):

    # Create a scatter plot
    plt.figure(figsize=(16, 9))  # Adjust the figure size as needed
    for index, row in df.iterrows():
        x = row['resnet_countours']['x']
        y = row['resnet_countours']['y']
        plt.scatter(x, y, c='blue', s=50, alpha=0.6)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Ball positions')
    plt.yticks(range(0, 1080 + 120, 120))
    plt.xticks(range(0, 1920 + 120, 120), rotation=45)
    plt.ylim((1080, 0))
    plt.xlim((0, 1920))
    plt.title('Ball positions')
    plt.grid(True)
    
    if save_filename is not None:
        plt.savefig(save_filename, bbox_inches='tight')
        print(f"Plot saved as {save_filename}")
    
    plt.show()

create_polar_bar_chart_with_transparency(angles_degrees=directions_df['theta'], num_bins=int(360/5), transparency=0.7, save_path='polar_test_bar.png')
plot_surface_points(main_df, save_filename='ball_positions.png')

Code to compute player positions heatmap

In [ ]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


jsonl_file = 'players_position.jsonl'

with open(jsonl_file, 'r') as f:
    lines = f.readlines()
data = []
for line in lines:
    json_data = json.loads(line)
    data.append(pd.DataFrame(json_data))
df = pd.concat(data)



# Create a 2D density plot using seaborn
plt.figure(figsize=(16, 9))  # Adjust the figure size as needed
sns.kdeplot(data=df, x='x', y='y', cmap='viridis', fill=True)
plt.ylim((1080, 0))
plt.xlim((0, 1920))
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.yticks(range(0, 1080 + 120, 120))
plt.xticks(range(0, 1920 + 120, 120), rotation=45)
plt.title('Player position density')
plt.savefig('density_player.png')
plt.show()